In [ ]:
%matplotlib inline
from deepdecoder.dcgan_train import *
from beras.util import tile
from beras.gan import GAN
import matplotlib.pyplot as plt
import numpy as np
from deepdecoder.mask_loss import mask_loss, mask_loss_sobel, mask_loss_mse, mask_loss_adaptive_mse
from deepdecoder.utils import masks, binary_mask
from beesgrid import MASK
from keras.optimizers import SGD, Adam, RMSprop

import pylab
pylab.rcParams['figure.figsize'] = (18, 18)

In [ ]:
gan = load_gan()
# gan.load_weights("lapgan/models/dcgan_g64_d32/{}.hdf5")

In [ ]:
X = loadRealData("/home/leon/repos/deeplocalizer_data/data/lapgan/lapgan.txt")

In [ ]:
def visualise_tiles(images):
    tiled_fakes = tile(images)
    plt.imshow(tiled_fakes[0], cmap='gray')
    plt.show()
    
def bw_mask(mask):
    bw = 0.5*np.ones_like(mask)
    bw[mask > MASK["IGNORE"]] = 1.
    bw[mask < MASK["BACKGROUND_RING"]] = 0
    return bw

def zip_visualise_tiles(xs, ys):
    assert len(xs) == len(ys)
    tiles = []
    for i in range(len(xs)):
        tiles.append(xs[i])
        tiles.append(ys[i])
    visualise_tiles(tiles)
    


In [ ]:
print("Compiling...")
start = time.time()
optimizer = lambda: Adam(lr=0.0002, beta_1=0.5)
gan.compile(optimizer(), optimizer(), gan_regulizer=GAN.L2Regularizer())
print("Done Compiling in {0:.2f}s".format(time.time() - start))

In [ ]:
try:
    visualise_tiles(gan.generate())
    nb_epoches = 250
    for i in range(nb_epoches):
        print(i)
        gan.fit(X, batch_size=128, verbose=1, nb_epoch=1)
        if i % 5 == 0 or i < 15: 
            visualise_tiles(gan.generate())
finally:
    output_dir = "models/dcgan_g64_d32"
    os.makedirs(output_dir, exist_ok=True)
    print("Saving model to {}".format(os.path.realpath(output_dir)))
    output_dir = output_dir + "/{}.hdf5"
    gan.save_weights(output_dir, overwrite=True)



In [ ]:
import numpy as np
arr = np.arange(len(X))
np.random.shuffle(arr)
visualise_tiles(X[arr[:128]])

In [ ]:
shp = gan.z_shape[1:]
z_point = lambda: np.random.uniform(-1, 1, shp)
for i in range(2):
    out = gan.interpolate(z_point(), z_point())
    visualise_tiles(out)

In [ ]:
print("Compiling...")
start = time.time()
loss_fn = lambda t, p: mask_loss_adaptive_mse(t, p).loss
gan.compile_optimize_image(Adam(), loss_fn)
print("Done Compiling in {0:.2f}s".format(time.time() - start))

In [ ]:
masks_idx, mask_bits, mask_grids = next(masks(gan.batch_size-1))
opt_z = None

In [ ]:
if 'opt_z' not in globals():
    opt_z = np.random.uniform(-1, 1, gan.z_shape)
duration = 0
for i in range(20):
    zip_visualise_tiles(bw_mask(masks_idx), gan.generate(opt_z))
    iterations = 256
    start = time.time()
    opt_images, opt_z = gan.optimize_image(masks_idx, iterations, z_start=opt_z, verbose=1)
    duration += time.time() - start
    if i % 5:
        zip_visualise_tiles(bw_mask(masks_idx), opt_images)
        jjjjj
zip_visualise_tiles(bw_mask(masks_idx), opt_images)

print("Done Optimizing in {0:.2f}s".format(duration))

In [ ]:
z = np.zeros(gan.z_shape, dtype=np.float32)
z[0] = gan.random_z_point()
for i in range(1, len(z), 2):
    z[i] = gan.random_z_point()
    z[i+1] = np.max(z[0] + 0.5 * np.ones(z[0].shape)
    z[i] = np.clip(z[i], -1, 1)
visualise_tiles(gan.generate(z))

In [ ]:
for _ in range(10):
    visualise_tiles(gan.neighborhood(opt_z[5], std=0.30))

In [ ]:
z_point = opt_z[0]
for i in range(50):
    print(i)
    z_min = np.copy(z_point)
    z_max = np.copy(z_point)
    z_min[i] = -1
    z_max[i] = 1
    visualise_tiles(gan.interpolate(z_min, z_max))

In [ ]:
print(len(masks_idx))
print(len(opt_images))